# レッスン 04  Attentionメカニズム

深層学習の歴史において、2017年の「Attention is All You Need」論文は転換点となりました。それまでのRNNやLSTMが持っていた「順次処理」という制約から解放され、並列処理可能で長距離依存関係を効果的に学習できる新しいアーキテクチャが誕生したのです。人間が文章を理解する過程を考えてみましょう。

「昨日公園で見た桜がとても美しかった」という文を読むとき、「美しかった」という形容詞を理解するために、私たちの脳は自動的に「桜」に注意を向けます。さらに「昨日」「公園」という文脈情報も同時に処理しています。

Attentionメカニズムは、まさにこの「選択的注意」の仕組みを数学的に実装したものです。

本講義では、単純な行列演算から始めて、段階的にSelf-Attentionの実装まで到達します。各ステップで「なぜこの操作が必要なのか」「どのような問題を解決しているのか」を理解しながら進めていきましょう。

## Section 1: 行列積の基礎 - 重み付き集約

Attentionを理解する第一歩として、「行列積が重み付き平均を計算できる」という基本原理から始めます。これは一見単純ですが、Attention機構全体を支える重要な数学的基盤です。

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

再現性のためのシード設定

In [9]:
torch.manual_seed(42)

3x3の下三角行列を作成

In [10]:
a = torch.tril(torch.ones(3, 3))
print("下三角行列（正規化前）:")
print(a)

下三角行列（正規化前）:
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])


各行を正規化（行の合計を1にする）

In [11]:
a = a / torch.sum(a, dim=1, keepdim=True)
print("\n正規化後の重み行列:")
print(a)


正規化後の重み行列:
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])


ランダムな値行列を生成と重み付き集約を実行

In [12]:
# ランダムな値行列を生成
b = torch.randint(0, 10, (3, 2)).float()
print("\n値行列b:")
print(b)

# 重み付き集約を実行
c = a @ b
print("\n結果c = a @ b:")
print(c)


値行列b:
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

結果c = a @ b:
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


### 直感的な理解を深める

この操作を具体的な例で考えてみましょう。bの各行を単語の特徴ベクトルだと想像してください。例えば、3つの単語「私は」「猫が」「好きだ」があるとします。

重み行列aの各行は、「その位置から見て、どの単語にどれだけ注目するか」を表しています。

第1行[1.0, 0.0, 0.0]は「最初の単語だけを見る」、第2行[0.5, 0.5, 0.0]は「最初の2つの単語を均等に見る」、第3行[0.33, 0.33, 0.33]は「全ての単語を均等に見る」という意味になります。

なぜ下三角行列を使うのでしょうか。これは「因果的マスキング（causal masking）」と呼ばれる重要な概念につながります。

言語モデルが文章を生成する際、各単語は自分より後の単語（未来の情報）を見ることができません。下三角行列はこの制約を自然に表現しているのです。

## Section 2: 実践的なバッチ処理 - Bag of Words方式の実装

実際の深層学習では、複数のサンプルを同時に処理する必要があります。ここでは、より実践的な設定での処理を見ていきます。

In [13]:
torch.manual_seed(1337)

# B: バッチサイズ（4つの独立したシーケンス）
# T: 時間ステップ数（各シーケンスは8トークン）
# C: チャネル数（各トークンは2次元ベクトル）
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

print(f"入力テンソルの形状: {x.shape}")
print(f"これは{B}個の文書、各文書は{T}個の単語、各単語は{C}次元のベクトル")

入力テンソルの形状: torch.Size([4, 8, 2])
これは4個の文書、各文書は8個の単語、各単語は2次元のベクトル


累積平均を計算する（ナイーブな実装）

In [14]:
xbow = torch.zeros((B, T, C))  # bow = "bag of words"の略

for b in range(B):
    for t in range(T):
        # 時刻tまでの全てのトークンを取得
        xprev = x[b, :t+1]  # shape: (t+1, C)
        
        # 平均を計算して格納
        xbow[b, t] = torch.mean(xprev, dim=0)
        
        # デバッグ用：最初のバッチの最初の3ステップを表示
        if b == 0 and t < 3:
            print(f"\nバッチ0, 時刻{t}:")
            print(f"  使用するトークン数: {t+1}")
            print(f"  平均値: {xbow[b, t]}")


バッチ0, 時刻0:
  使用するトークン数: 1
  平均値: tensor([ 0.1808, -0.0700])

バッチ0, 時刻1:
  使用するトークン数: 2
  平均値: tensor([-0.0894, -0.4926])

バッチ0, 時刻2:
  使用するトークン数: 3
  平均値: tensor([ 0.1490, -0.3199])


### この処理の意味を考える
各時刻で過去の全ての情報を平均化するこの処理には、深い意味があります。文章を読む際、私たちは読み進めるにつれて文脈を蓄積していきます。

「今日は天気が良い。公園に行こう。」という文章で、「公園」を理解する時には「今日」「天気が良い」という前の情報も考慮に入れています。

しかし、単純な平均には問題があります。全ての過去の情報を均等に扱うため、重要な情報もそうでない情報も同じ重みになってしまうのです。

これが、後で学ぶ「学習可能な重み」の必要性につながります。

## Section 3: 行列演算による効率化

Section 2のループ処理は理解しやすいですが、実用的ではありません。ここで、Section 1で学んだ行列積の知識を活用します。

In [15]:
# 効率的な実装：行列積を使用
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(dim=1, keepdim=True)

print("重み行列の形状:", wei.shape)
print("重み行列の一部（最初の4x4）:")
print(wei[:4, :4])

# バッチ処理を一度に実行
# PyTorchのブロードキャスティングにより、
# (T, T) @ (B, T, C) -> (B, T, C) が自動的に処理される
xbow2 = wei @ x

# 結果が同一であることを確認
print(f"\n二つの方法の結果は同じか: {torch.allclose(xbow, xbow2)}")

重み行列の形状: torch.Size([8, 8])
重み行列の一部（最初の4x4）:
tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])

二つの方法の結果は同じか: False


### 計算速度の比較（仮想的な例）

In [16]:
import torch
import time

# 実践的なサイズで比較（小規模版）
B, T, C = 16, 128, 256
x = torch.randn(B, T, C)
print(f"データサイズ: バッチ={B}, シーケンス長={T}, 特徴次元={C}")

# ===== 方法1: ループによる実装 =====
def compute_with_loop(x):
    B, T, C = x.shape
    result = torch.zeros_like(x)
    for b in range(B):
        for t in range(T):
            xprev = x[b, :t+1]
            result[b, t] = torch.mean(xprev, dim=0)
    return result

# ===== 方法2: 行列積による実装 =====
def compute_with_matrix(x):
    B, T, C = x.shape
    wei = torch.tril(torch.ones(T, T))
    wei = wei / wei.sum(dim=1, keepdim=True)
    return wei @ x

# 時間測定
print("\n処理時間の比較:")
start = time.time()
result_loop = compute_with_loop(x)
time_loop = time.time() - start
print(f"ループ版: {time_loop:.4f}秒")

start = time.time()
result_matrix = compute_with_matrix(x)
time_matrix = time.time() - start
print(f"行列積版: {time_matrix:.4f}秒")

# 結果の検証
print(f"\n計算結果は一致: {torch.allclose(result_loop, result_matrix)}")
print(f"高速化率: {time_loop/time_matrix:.1f}倍")

# 大規模データでの推定
B_large, T_large = 32, 512
scale_factor = (B_large/B) * (T_large/T) * (T_large/T)
print(f"\nフルサイズ（B={B_large}, T={T_large}）での推定:")
print(f"ループ版: 約{time_loop * scale_factor:.1f}秒")
print(f"行列積版: 約{time_matrix * (T_large/T)**2:.3f}秒")

データサイズ: バッチ=16, シーケンス長=128, 特徴次元=256

処理時間の比較:
ループ版: 0.0178秒
行列積版: 0.0017秒

計算結果は一致: False
高速化率: 10.7倍

フルサイズ（B=32, T=512）での推定:
ループ版: 約0.6秒
行列積版: 約0.026秒


## Section 4: Self-Attentionの実装: 動的な注意機構
### 概念の導入と直感的理解

Self-Attentionの核心は、入力データから三つの異なる表現を作り出すことから始まります。これは情報検索システムからの美しいアナロジーです。

図書館で本を探す場面を想像してください。あなたが持っている「検索キーワード」（Query）、各本についている「インデックスカード」（Key）、そして「本の実際の内容」（Value）という三要素の相互作用と考えることができます。

In [17]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # バッチ=4、時系列=8、チャネル=32
x = torch.randn(B, T, C)

print(f"入力データ x の形状: {x.shape}")
print(f"これは{B}個の文書、各{T}トークン、各トークンは{C}次元の表現")

# Attention Headのサイズを定義
# なぜ元の32次元から16次元に削減するのか？
# 1. 計算効率の向上（パラメータ数の削減）
# 2. 過学習の防止（表現力の適切な制限）
# 3. Multi-Head Attentionで複数のヘッドを使う準備
head_size = 16

# 三つの線形変換層を定義（これらは学習可能なパラメータ）
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

# 各表現を計算
k = key(x)    # (B, T, 16) - 各トークンの「キー」表現
q = query(x)  # (B, T, 16) - 各トークンの「クエリ」表現
v = value(x)  # (B, T, 16) - 各トークンの「バリュー」表現

print(f"\nQuery形状: {q.shape} - 『私は何を探しているか』")
print(f"Key形状: {k.shape} - 『私は何についての情報を持っているか』")
print(f"Value形状: {v.shape} - 『私が実際に提供できる情報』")

# 具体例で理解を深める
print("\n例：「猫が魚を食べる」という文を処理する場合")
print("- '食べる'のQuery: 『誰が何を食べるのか知りたい』")
print("- '猫'のKey: 『私は動作の主体についての情報を持っている』")
print("- '猫'のValue: 『実際の猫に関する特徴情報』")

入力データ x の形状: torch.Size([4, 8, 32])
これは4個の文書、各8トークン、各トークンは32次元の表現

Query形状: torch.Size([4, 8, 16]) - 『私は何を探しているか』
Key形状: torch.Size([4, 8, 16]) - 『私は何についての情報を持っているか』
Value形状: torch.Size([4, 8, 16]) - 『私が実際に提供できる情報』

例：「猫が魚を食べる」という文を処理する場合
- '食べる'のQuery: 『誰が何を食べるのか知りたい』
- '猫'のKey: 『私は動作の主体についての情報を持っている』
- '猫'のValue: 『実際の猫に関する特徴情報』


この三つの表現への分離は、なぜ必要なのでしょうか。

それは、「何を探すか」「何として見つかるか」「何を提供するか」を分離することで、より柔軟で表現力豊かな注意メカニズムを実現できるからです。

同じトークンでも、文脈によって異なる役割を果たすことができるのです。

### 相性スコアの計算と因果的制約の実装

次に、各QueryとKeyの間の「相性」を計算し、どのトークンにどれだけ注目すべきかを決定します。この段階で、言語モデル特有の重要な制約「未来を見ない」も実装します。

In [18]:
# ステップ1: Attentionスコアの計算
# QueryとKeyの内積により、各位置間の関連性スコアを計算
wei = q @ k.transpose(-2, -1)  
# (B, T, 16) @ (B, 16, T) -> (B, T, T)

print(f"Attentionスコア行列の形状: {wei.shape}")
print("wei[b,i,j] = 位置iのクエリと位置jのキーの相性スコア")

# 重要：スケーリングファクターの適用
# なぜ√head_sizeで割るのか？
# 内積の値が大きくなりすぎると、Softmaxが極端な値（0か1）を出力しやすくなる
# これは勾配消失を引き起こし、学習を困難にする
scale_factor = head_size ** -0.5
wei = wei * scale_factor
print(f"\nスケーリングファクター: 1/√{head_size} = {scale_factor:.4f}")

# ステップ2: Causal Maskingの適用
# 下三角行列を作成（未来の情報を見ない制約）
tril = torch.tril(torch.ones(T, T))
print(f"\nCausalマスク（最初の5x5）:")
print(tril[:5, :5])

# マスキングの実装：未来の位置を負の無限大に設定
# なぜ-infなのか？exp(-inf) = 0となり、Softmax後に完全に0になる
wei_before_mask = wei.clone()
wei = wei.masked_fill(tril == 0, float('-inf'))

print("\nマスク適用前後の比較（最初のバッチ、最初の3x3）:")
print("適用前:")
print(wei_before_mask[0, :3, :3])
print("\n適用後:")
print(wei[0, :3, :3])
print("（-infは未来の位置を完全に無視することを意味する）")

# 視覚的な理解のための例
print("\n時系列での解釈:")
print("位置0: 自分だけを見る")
print("位置1: 位置0と自分を見る") 
print("位置2: 位置0, 1と自分を見る")
print("...これが言語生成での自己回帰的な性質を実現")

Attentionスコア行列の形状: torch.Size([4, 8, 8])
wei[b,i,j] = 位置iのクエリと位置jのキーの相性スコア

スケーリングファクター: 1/√16 = 0.2500

Causalマスク（最初の5x5）:
tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

マスク適用前後の比較（最初のバッチ、最初の3x3）:
適用前:
tensor([[-0.4407, -0.3253,  0.1413],
        [-0.8334, -0.4139,  0.0260],
        [-0.2557, -0.3152,  0.0191]], grad_fn=<SliceBackward0>)

適用後:
tensor([[-0.4407,    -inf,    -inf],
        [-0.8334, -0.4139,    -inf],
        [-0.2557, -0.3152,  0.0191]], grad_fn=<SliceBackward0>)
（-infは未来の位置を完全に無視することを意味する）

時系列での解釈:
位置0: 自分だけを見る
位置1: 位置0と自分を見る
位置2: 位置0, 1と自分を見る
...これが言語生成での自己回帰的な性質を実現


このマスキング処理により、モデルは訓練時と推論時で一貫した動作をします。

各トークンは、自分より前のトークンだけを参照して次の表現を計算するため、左から右への文章生成が自然に実現されるのです。

### 確率分布への変換と重み付き和の計算

最後のステップでは、スコアを確率分布に変換し、その重みを使ってValue表現を集約します。これにより、各位置が「自分が注目すべき他の位置の情報」を選択的に取り込んだ新しい表現を得ます。

In [19]:
# ステップ3: Softmaxによる正規化
# 各行のスコアを確率分布に変換（合計が1になる）
wei_before_softmax = wei.clone()
wei = F.softmax(wei, dim=-1)

print("Softmax適用前後の比較（最初のバッチ、位置2の注意重み）:")
print(f"適用前のスコア: {wei_before_softmax[0, 2, :5]}")
print(f"適用後の確率: {wei[0, 2, :5]}")
print(f"確率の合計: {wei[0, 2, :3].sum().item():.4f}")  # 見える範囲の合計は1

# Attention重みの可視化（仮想的な例）
print("\n実際のAttention重みパターンの例:")
example_weights = wei[0, :4, :4]
for i in range(4):
    weights_str = [f"{w:.3f}" for w in example_weights[i, :i+1]]
    print(f"位置{i}: [{', '.join(weights_str)}]")

# ステップ4: 最終的な重み付き和の計算
out = wei @ v  # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)

print(f"\n最終出力の形状: {out.shape}")
print("各位置が、注意重みに基づいて他の位置のValue情報を統合")

# 計算の意味を具体例で理解
print("\n具体例：位置3の新しい表現の計算")
print("out[b,3] = ")
print("  wei[b,3,0] * v[b,0] +  # 位置0への注意重み × 位置0の値")
print("  wei[b,3,1] * v[b,1] +  # 位置1への注意重み × 位置1の値")
print("  wei[b,3,2] * v[b,2] +  # 位置2への注意重み × 位置2の値")
print("  wei[b,3,3] * v[b,3]    # 自分への注意重み × 自分の値")

# Self-Attentionの完全な処理を確認
print("\n=== Self-Attentionの処理完了 ===")
print(f"入力: {x.shape} -> Query/Key/Value変換 -> スコア計算 -> ")
print(f"マスキング -> Softmax -> 重み付き和 -> 出力: {out.shape}")

Softmax適用前後の比較（最初のバッチ、位置2の注意重み）:
適用前のスコア: tensor([-0.2557, -0.3152,  0.0191,    -inf,    -inf], grad_fn=<SliceBackward0>)
適用後の確率: tensor([0.3069, 0.2892, 0.4039, 0.0000, 0.0000], grad_fn=<SliceBackward0>)
確率の合計: 1.0000

実際のAttention重みパターンの例:
位置0: [1.000]
位置1: [0.397, 0.603]
位置2: [0.307, 0.289, 0.404]
位置3: [0.323, 0.218, 0.244, 0.215]

最終出力の形状: torch.Size([4, 8, 16])
各位置が、注意重みに基づいて他の位置のValue情報を統合

具体例：位置3の新しい表現の計算
out[b,3] = 
  wei[b,3,0] * v[b,0] +  # 位置0への注意重み × 位置0の値
  wei[b,3,1] * v[b,1] +  # 位置1への注意重み × 位置1の値
  wei[b,3,2] * v[b,2] +  # 位置2への注意重み × 位置2の値
  wei[b,3,3] * v[b,3]    # 自分への注意重み × 自分の値

=== Self-Attentionの処理完了 ===
入力: torch.Size([4, 8, 32]) -> Query/Key/Value変換 -> スコア計算 -> 
マスキング -> Softmax -> 重み付き和 -> 出力: torch.Size([4, 8, 16])


このメカニズムの素晴らしさを改めて考えてみましょう。従来の固定重みでは不可能だった「文脈に応じた動的な情報選択」が可能になりました。

例えば、「銀行でお金を下ろす」と「川の銀行で釣りをする」という二つの文で、同じ「銀行」という単語でも全く異なる注意パターンを持ちます。

前者では「お金」「下ろす」により強く注目し、金融機関としての意味を活性化させます。後者では「川」「釣り」に注目し、地理的な意味を活性化させます。

この文脈依存の意味理解こそが、現代のAIが人間のような言語理解を示す理由なのです。

さらに、この仕組みは完全に微分可能であるため、誤差逆伝播によって学習できます。モデルは大量のテキストから、どのような文脈でどのトークンに注目すべきかを自動的に学習していくのです。

これがTransformerアーキテクチャの革新性であり、ChatGPTのような大規模言語モデルの基盤となっている技術なのです。

## 参考コード

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 1000
learning_rate = 1e-3
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

print("Running on " + device)

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers
decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))
